In [4]:
header=["outlook","temperature","humidity","wind","decision"]

training_data=[
['sunny,hot','high','weak','no'],
['sunny','hot','high','strong','no'],
['overcast','hot','high','weak','yes'],
['rain','mild','high','weak','yes'],
['rain','cool','normal','weak','yes'],
['rain','cool','normal','strong','no'],
['overcast','cool','normal','strong','yes'],
['sunny','mild','high','weak','no'],
['sunny','cool','normal','weak','yes'],
['rain','mild','normal','weak','yes'],
['sunny','mild','normal','strong','yes'],
['overcast','mild','high','strong','yes'],
['overcast','hot','normal','weak','yes'],
['rain','mild','high','strong','no'],
]

In [5]:
def unique_vals(Data,col):
    return set([row[col] for row in Data])

In [6]:
unique_vals(training_data,1)

{'cool', 'high', 'hot', 'mild'}

In [7]:
def class_counts(Data):
    counts={}
    for row in Data:
        label = row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts

In [8]:
class_counts(training_data)

{'no': 5, 'yes': 9}

In [9]:
def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

In [10]:
is_numeric(50)

True

In [11]:
class Question():
    def __init__(self,column,value):
        self.column=column
        self.value=value
    def match(self,example):
        val=example[self.column]
        if is_numeric(val):
            return val ==self.value
        else:
            return val==self.value
    def __repr__(self):
        condition= "=="
        if is_numeric(self.value):
            condition=">="
        return "IS %s %s %s?" % (header[self.column],condition,str(self.value))
        

In [12]:
Question(0,'red')

IS outlook == red?

In [13]:
def partition(rows,question):
    true_rows,false_rows=[], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows

In [14]:
true_rows, false_rows = partition(training_data, Question(1,3))

In [15]:
false_rows

[['sunny,hot', 'high', 'weak', 'no'],
 ['sunny', 'hot', 'high', 'strong', 'no'],
 ['overcast', 'hot', 'high', 'weak', 'yes'],
 ['rain', 'mild', 'high', 'weak', 'yes'],
 ['rain', 'cool', 'normal', 'weak', 'yes'],
 ['rain', 'cool', 'normal', 'strong', 'no'],
 ['overcast', 'cool', 'normal', 'strong', 'yes'],
 ['sunny', 'mild', 'high', 'weak', 'no'],
 ['sunny', 'cool', 'normal', 'weak', 'yes'],
 ['rain', 'mild', 'normal', 'weak', 'yes'],
 ['sunny', 'mild', 'normal', 'strong', 'yes'],
 ['overcast', 'mild', 'high', 'strong', 'yes'],
 ['overcast', 'hot', 'normal', 'weak', 'yes'],
 ['rain', 'mild', 'high', 'strong', 'no']]

In [16]:
def gini(rows):
    counts = class_counts(rows)
    impurity=1
    for lbl in counts:
        prob_of_lbl=counts[lbl]/float(len(rows))
        impurity -= prob_of_lbl **2
    return impurity

In [17]:
x=[['apple'],['apple'],['abc'],['abc']]
gini(training_data)


0.4591836734693877

In [18]:
def info_gain(left,right,current_uncertainty):
    """Information Gain=GINI-P(TRUE)*GINI(TRUE)-P(FALSE)*GINI(FALSE)"""
    p=float(len(left))/(len(left)+len(right))
    q=float(len(right))/(len(left)+len(right))
    return current_uncertainty - p *gini(left)-q*gini(right)

In [19]:
true_rows,false_rows = partition(training_data,Question(1,1))

In [20]:
info_gain(true_rows,false_rows,0.639)

0.1798163265306123

In [21]:
def find_best_split(rows):
    best_gain=0 #Keep track of best information gain
    best_question=None #keep track of the feature/value
    current_uncertainty=gini(rows)
    n_features=len(rows[0])-1 # number of features=number of total columns- number of label(1)
    for col in range(n_features):#for each new feature
        values=set([row[col] for row in rows])
        for val in values:
            question=Question(col,val)
            true_rows,false_rows=partition(rows,question)
            if len(true_rows)==0 or len(false_rows)==0:
                continue
            gain=info_gain(true_rows,false_rows,current_uncertainty)
            if gain>=best_gain:
                best_gain,best_question=gain,question
    return best_gain,best_question

In [22]:
best_gain,best_question=find_best_split(training_data)
print(best_question)
print(best_gain)

IS outlook == overcast?
0.102040816327


In [23]:
class leaf:
    def __init__(self,rows):
        self.predictions = class_counts(rows)

In [24]:
class Decision_Node:
    """A Decision Node asks a question
    This holds a reference to the question, and to the two nodes"""
    def __init__(self,question,true_branch,false_branch):
        self.question=question
        self.true_branch =true_branch
        self.false_branch = false_branch

In [25]:
def build_tree(rows):
    gain,question = find_best_split(rows)
    if gain==0:
        return leaf(rows)
    true_rows,false_rows =partition(rows,question)
    true_branch=build_tree(true_rows)
    false_branch=build_tree(false_rows)
    return Decision_Node(question,true_branch,false_branch)

In [26]:
def print_tree(node,spacing=" "):
    #Base Case: We have reached the leaf
    if isinstance(node,leaf):
        print(spacing+"Predict",node.predictions)
        return
    #Print the question at this node
    print(spacing+str(node.question))
    
    #call this function recursively on the true branch
    print(spacing+ '---> TRUE:' )
    print_tree(node.true_branch,spacing + " ")
    
    #call this function recursively of false branch
    print(spacing+'---> False:')
    print_tree(node.false_branch,spacing+" ")
    

In [28]:
my_tree=build_tree(training_data)
print_tree(my_tree)

 IS outlook == overcast?
 ---> TRUE:
('  Predict', {'yes': 4})
 ---> False:
  IS humidity == normal?
  ---> TRUE:
   IS wind == weak?
   ---> TRUE:
('    Predict', {'yes': 3})
   ---> False:
    IS temperature == cool?
    ---> TRUE:
('     Predict', {'no': 1})
    ---> False:
('     Predict', {'yes': 1})
  ---> False:
   IS outlook == rain?
   ---> TRUE:
    IS wind == weak?
    ---> TRUE:
('     Predict', {'yes': 1})
    ---> False:
('     Predict', {'no': 1})
   ---> False:
('    Predict', {'no': 3})
